In [5]:
### Import von library

import pandas as pd
import xml.etree.ElementTree as ET
import requests
import psycopg2
import os
import time
from sqlalchemy import create_engine
from dotenv import load_dotenv
from lxml import etree

# Umgebungsvariablen laden (sollten identisch zu den anderen Skripts sein)
load_dotenv()


dbname = os.getenv('DB_NAME', 'DB_NAME')
user = os.getenv('DB_USER', 'DB_USER')
password = os.getenv('DB_PASSWORD', 'DB_PASSWORD')
host = os.getenv('DB_HOST', 'DB_HOST')
schema = 'Source'
start_time = time.time()



### funktion definieren, um Daten aus diesem ultra hässlichen XML zu extrahieren
def fetch_eurostat_data(dataset_code):
    url = f"https://ec.europa.eu/eurostat/api/dissemination/sdmx/3.0/data/dataflow/ESTAT/{dataset_code}/1.0?compress=false"
    print(url)

    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to fetch data for {dataset_code}. Status code: {response.status_code}")
        return None

    root = etree.fromstring(response.content)

    # die Logik ist vermutlich korrekt aber ich bin mir nicht sicher, ob findall hier besser wäre
    structured_data = []
    for series in root.findall('.//Series', namespaces={}):
        series_data = series.attrib
        for obs in series.findall('.//Obs', namespaces={}):
            obs_data = obs.attrib
            record = {**series_data, **obs_data}
            structured_data.append(record) ### Alle Daten werden extrahiert und untereinandergehängt

    df = pd.DataFrame(structured_data)

    if df.empty:
        print("Bitte hinterfrage dein Leben, vermutlich ist der Namespace nicht korrekt")
    else:
        print(df.head())

    return df

data = fetch_eurostat_data('gov_10a_main')



# Daten analog zu bisherigen Skripten auf die lokale Datenbank laden
# Offenbar kann postgres nicht mi nans umgehen, keine Ahnung warum aber in diesem Skript scheinen nan akzeptabe
# Also Aufgabe für Charlotte: NANs rausfiltern


def upload_to_postgres(df, table_name):
    engine = create_engine(f'postgresql://{user}:{password}@{host}/{dbname}')
    df.columns = [col.replace(' ', '_').lower() for col in df.columns] ### Spalten in Tabelle haben gleichen Namen wie Spalten in Dataframe
    df = df.apply(pd.to_numeric, errors='ignore')
    df.to_sql(table_name, engine, schema=schema, if_exists='replace', index=False, chunksize=500)


dataset_mapping = {
    'gov_10a_main': 'gov_10a_main_table' ### Du kannst die Tabelle natürlich nennen wie du lustig bist
}

### funktion, die das Beste aus allen Welten zusammenfügt
for dataset_code, table_name in dataset_mapping.items():
    print(f"\nDataset: {dataset_code}")
    data = fetch_eurostat_data(dataset_code)
    if data is not None:
        upload_to_postgres(data, table_name)

### Runtime berechnen
end_time = time.time()
runtime = end_time - start_time
print(f"Script runtime: {runtime} seconds")


https://ec.europa.eu/eurostat/api/dissemination/sdmx/3.0/data/dataflow/ESTAT/gov_10a_main/1.0?compress=false
  freq geo     na_item sector     unit TIME_PERIOD OBS_VALUE OBS_FLAG
0    A  BE  D3REC_S212     S1  MIO_EUR        1995    1068.5      NaN
1    A  BE  D3REC_S212     S1  MIO_EUR        1996     864.6      NaN
2    A  BE  D3REC_S212     S1  MIO_EUR        1997     824.2      NaN
3    A  BE  D3REC_S212     S1  MIO_EUR        1998     738.1      NaN
4    A  BE  D3REC_S212     S1  MIO_EUR        1999     857.6      NaN

Dataset: gov_10a_main
https://ec.europa.eu/eurostat/api/dissemination/sdmx/3.0/data/dataflow/ESTAT/gov_10a_main/1.0?compress=false
  freq geo     na_item sector     unit TIME_PERIOD OBS_VALUE OBS_FLAG
0    A  BE  D3REC_S212     S1  MIO_EUR        1995    1068.5      NaN
1    A  BE  D3REC_S212     S1  MIO_EUR        1996     864.6      NaN
2    A  BE  D3REC_S212     S1  MIO_EUR        1997     824.2      NaN
3    A  BE  D3REC_S212     S1  MIO_EUR        1998     738.